<a href="https://colab.research.google.com/github/28Aarya/FHI-score-for-stability/blob/main/Financial_Health_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install yfinance

In [27]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [28]:
# Fetch the stocks and relevant data
import yfinance as yf
import pandas as pd

stocks = ['AAPL', 'MSFT', 'NVDA', 'NOW', 'NOK', 'INTC', 'TSLA', 'ZM']

def fetch_metrics(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info

    # Define metrics inside the function
    metrics = {
        'ticker': ticker,
        'currentRatio': info.get('currentRatio', None),
        'debtToEquity': info.get('debtToEquity', None),
        'returnOnEquity': info.get('returnOnEquity', None),
        'priceToBook': info.get('priceToBook', None),
        'earningsGrowth': info.get('earningsGrowth', None),
        'dividendYield': info.get('dividendYield', None),
        'beta': info.get('beta', None)
    }

    return metrics

financial_data = [fetch_metrics(stock) for stock in stocks]
financial_df = pd.DataFrame(financial_data)
financial_df.head(8)


,ticker,currentRatio,debtToEquity,returnOnEquity,priceToBook,earningsGrowth,dividendYield,beta
0,AAPL,0.867,209.059,1.57413,61.653835,-0.341,0.0044,1.240
1,MSFT,1.301,33.657,0.35604,10.744321,0.104,0.0080,0.904
2,NVDA,4.269,17.221,1.23767,79.088300,1.680,0.0003,1.657
3,NOW,1.134,24.166,0.16221,23.614420,0.769,NaN,0.986
4,NOK,1.676,22.538,0.04488,1.131155,0.506,0.0337,0.737
5,INTC,1.312,47.906,-0.15597,1.080350,NaN,0.0224,1.026
6,TSLA,1.844,18.078,0.20389,16.120333,0.170,NaN,2.295
7,ZM,4.559,0.751,0.11275,3.265535,0.173,NaN,-0.055


In [29]:
financial_df.isnull().sum()

,0
ticker,0
currentRatio,0
debtToEquity,0
returnOnEquity,0
priceToBook,0
earningsGrowth,1
dividendYield,3
beta,0


In [33]:
print(financial_df.loc[financial_df['dividendYield'].isnull(), ['ticker', 'dividendYield']])
print(financial_df.loc[financial_df['earningsGrowth'].isnull(), ['ticker', 'earningsGrowth']])

  ticker  dividendYield
3    NOW            NaN
6   TSLA            NaN
7     ZM            NaN
  ticker  earningsGrowth
5   INTC             NaN


In [34]:
financial_df['dividendYield'].fillna(00, inplace=True) #cause company doesnt give out dividends to share holders
financial_df.head(8)

<ipython-input-34-71849bc3da29>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  financial_df['dividendYield'].fillna(00, inplace=True) #cause company doesnt give out dividends to share holders


,ticker,currentRatio,debtToEquity,returnOnEquity,priceToBook,earningsGrowth,dividendYield,beta
0,AAPL,0.867,209.059,1.57413,61.653835,-0.341,0.0044,1.240
1,MSFT,1.301,33.657,0.35604,10.744321,0.104,0.0080,0.904
2,NVDA,4.269,17.221,1.23767,79.088300,1.680,0.0003,1.657
3,NOW,1.134,24.166,0.16221,23.614420,0.769,0.0000,0.986
4,NOK,1.676,22.538,0.04488,1.131155,0.506,0.0337,0.737
5,INTC,1.312,47.906,-0.15597,1.080350,NaN,0.0224,1.026
6,TSLA,1.844,18.078,0.20389,16.120333,0.170,0.0000,2.295
7,ZM,4.559,0.751,0.11275,3.265535,0.173,0.0000,-0.055


In [35]:
# Earnings per share for 2023 and 2024 as per the internet for INTC
eps_2024 = -0.46
eps_2023 = 0.54

# Calculate earnings growth
earnings_growth = (eps_2024 - eps_2023) / eps_2023
financial_df['earningsGrowth'].fillna(earnings_growth, inplace=True)
financial_df.head(6)

<ipython-input-35-fb77b77d48d1>:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  financial_df['earningsGrowth'].fillna(earnings_growth, inplace=True)


,ticker,currentRatio,debtToEquity,returnOnEquity,priceToBook,earningsGrowth,dividendYield,beta
0,AAPL,0.867,209.059,1.57413,61.653835,-0.341000,0.0044,1.240
1,MSFT,1.301,33.657,0.35604,10.744321,0.104000,0.0080,0.904
2,NVDA,4.269,17.221,1.23767,79.088300,1.680000,0.0003,1.657
3,NOW,1.134,24.166,0.16221,23.614420,0.769000,0.0000,0.986
4,NOK,1.676,22.538,0.04488,1.131155,0.506000,0.0337,0.737
5,INTC,1.312,47.906,-0.15597,1.080350,-1.851852,0.0224,1.026


In [44]:
#standardise the data to make it more comparable

std_columns= ['currentRatio',	'debtToEquity',	'returnOnEquity',	'priceToBook',	'earningsGrowth',	'dividendYield',	'beta'] #dividend payout ratio instead
financial_df[std_columns] = financial_df[std_columns].apply(lambda x: (x - x.mean()) / x.std())
financial_df.head(8)

,ticker,currentRatio,debtToEquity,returnOnEquity,priceToBook,earningsGrowth,dividendYield,beta,FHI,Performance
0,AAPL,-0.864646,2.423681,1.829481,1.248912,-0.488585,-0.330681,0.206331,0.808936,Overperforming
1,MSFT,-0.565220,-0.194253,-0.138821,-0.466421,-0.046803,-0.047240,-0.284481,-0.287903,Underperforming
2,NVDA,1.482473,-0.439566,1.285798,1.836344,1.517803,-0.653488,0.815464,0.828725,Overperforming
3,NOW,-0.680437,-0.335910,-0.452030,-0.032779,0.613390,-0.677108,-0.164700,-0.362377,Underperforming
4,NOK,-0.306498,-0.360208,-0.641622,-0.790325,0.352291,1.976212,-0.528427,-0.167605,Underperforming
5,INTC,-0.557630,0.018418,-0.966174,-0.792037,-1.988514,1.086523,-0.106270,-0.502754,Underperforming
6,TSLA,-0.190591,-0.426775,-0.384679,-0.285283,0.018720,-0.677108,1.747423,-0.238273,Underperforming
7,ZM,1.682550,-0.685387,-0.531952,-0.718410,0.021698,-0.677108,-1.685341,-0.078748,Underperforming


In [37]:
financial_df.describe()

,currentRatio,debtToEquity,returnOnEquity,priceToBook,earningsGrowth,dividendYield,beta
count,8.000000e+00,8.000000,8.000000e+00,8.000000,8.000000,8.000000e+00,8.000000e+00
mean,4.163336e-17,0.000000,-5.551115e-17,0.000000,0.000000,5.551115e-17,1.942890e-16
std,1.000000e+00,1.000000,1.000000e+00,1.000000,1.000000,1.000000e+00,1.000000e+00
min,-8.646464e-01,-0.685387,-9.661742e-01,-0.792037,-1.988514,-6.771084e-01,-1.685341e+00
25%,-5.940239e-01,-0.429973,-5.593693e-01,-0.736389,-0.157248,-6.771084e-01,-3.454676e-01
50%,-4.320645e-01,-0.348059,-4.183545e-01,-0.375852,0.020209,-4.920846e-01,-1.354846e-01
75%,2.276747e-01,-0.141086,2.173335e-01,0.287643,0.417566,2.362006e-01,3.586143e-01
max,1.682550e+00,2.423681,1.829481e+00,1.836344,1.517803,1.976212e+00,1.747423e+00


In [38]:
# Assign weights ficused on FHI for stability

weights = {
    'currentRatio': 0.25,
    'debtToEquity': 0.25,
    'returnOnEquity': 0.2,
    'priceToBook': 0.1,
    'earningsGrowth': 0.1,
    'dividendYield': 0.1,
    'beta': 0.05
}


In [39]:
# Function to calculate FHI for each stock
def calculate_fhi(row, weights):
    fhi_score = 0
    for metric, weight in weights.items():
        if pd.notnull(row[metric]):
            fhi_score += row[metric] * weight
    return fhi_score

# Apply the function to calculate FHI
financial_df['FHI'] = financial_df.apply(calculate_fhi, axis=1, weights=weights)
print(financial_df[['ticker', 'FHI']])


  ticker       FHI
0   AAPL  0.808936
1   MSFT -0.287903
2   NVDA  0.828725
3    NOW -0.362377
4    NOK -0.167605
5   INTC -0.502754
6   TSLA -0.238273
7     ZM -0.078748


In [40]:
financial_df.head(8)

,ticker,currentRatio,debtToEquity,returnOnEquity,priceToBook,earningsGrowth,dividendYield,beta,FHI
0,AAPL,-0.864646,2.423681,1.829481,1.248912,-0.488585,-0.330681,0.206331,0.808936
1,MSFT,-0.565220,-0.194253,-0.138821,-0.466421,-0.046803,-0.047240,-0.284481,-0.287903
2,NVDA,1.482473,-0.439566,1.285798,1.836344,1.517803,-0.653488,0.815464,0.828725
3,NOW,-0.680437,-0.335910,-0.452030,-0.032779,0.613390,-0.677108,-0.164700,-0.362377
4,NOK,-0.306498,-0.360208,-0.641622,-0.790325,0.352291,1.976212,-0.528427,-0.167605
5,INTC,-0.557630,0.018418,-0.966174,-0.792037,-1.988514,1.086523,-0.106270,-0.502754
6,TSLA,-0.190591,-0.426775,-0.384679,-0.285283,0.018720,-0.677108,1.747423,-0.238273
7,ZM,1.682550,-0.685387,-0.531952,-0.718410,0.021698,-0.677108,-1.685341,-0.078748


In [41]:
#calculate sector fhi, compare with the stocks for further data analysis
sector_avg_fhi = financial_df['FHI'].mean()

# Performance of stocks
financial_df['Performance'] = financial_df['FHI'].apply(lambda x: 'Overperforming' if x > sector_avg_fhi else 'Underperforming')

print(f"Sector Average FHI: {sector_avg_fhi:.2f}")
print(financial_df[['ticker', 'FHI', 'Performance']])



Sector Average FHI: 0.00
  ticker       FHI      Performance
0   AAPL  0.808936   Overperforming
1   MSFT -0.287903  Underperforming
2   NVDA  0.828725   Overperforming
3    NOW -0.362377  Underperforming
4    NOK -0.167605  Underperforming
5   INTC -0.502754  Underperforming
6   TSLA -0.238273  Underperforming
7     ZM -0.078748  Underperforming


In [ ]:
#detecting outliers for undervaluation and overvaluation- using z score(?)
'''def evaluate_valuation(row):
    if row['zscore'] <-1 and row['fhi'] > sector_avg['fhi']:
        return "Undervalued"
    elif row['zscore'] > 1 and row['fhi'] < sector_avg['fhi']:
        return "Overvalued"
    else:
        return "Fairly Valued"

financial_df['valuation'] = financial_df.apply(evaluate_valuation, axis=1)'''


In [ ]:
'''new_df= financial_df[['ticker', 'FHI', 'Performance', 'Valuation(?)']]
new_df'''

,ticker,FHI,Performance
0,AAPL,0.416225,Overperforming
1,MSFT,-0.229877,Underperforming
2,NVDA,0.411984,Overperforming
3,NOW,0.033862,Overperforming
4,NOK,-0.200523,Underperforming
5,INTC,-0.481599,Underperforming
6,TSLA,0.157838,Overperforming
7,ZM,-0.107910,Underperforming


In [ ]:
#data insights: solvency, liquidity and profitability
'''def financial_summary(row):
  solvency=
  liqiudity=
  profitability=
  return solvency, liqiudity, profitability

financial_df['summary']= financial_df.apply(financial_summary, axis=1)
'''

In [ ]:
#report BASED ON VALUATION,summary and fhi/ performance- use reportlabs or fpdf

In [ ]:
'''
#investment recommendation ^?
def stock_recommendation(row):
  if row['valuation']==
    return 'buy'
  if row ['valuation']==
    return 'sell'
  else:
    return 'hold'

financial_df['recommendation']= financial_df.apply(stock_recommendation, axis=1)
'''



In [ ]:
#prediction(?) with prophet